# 📦 Instalação de Dependências

Antes de começar, vamos instalar os pacotes necessários:

In [ ]:
!pip install duckdb pandas polars pyarrow -q

---

## 🎯 Bloco 1: Consultas Básicas

Uso mais simples do DuckDB: executar queries SQL diretamente.

In [ ]:
import duckdb

# Query simples
result = duckdb.sql("SELECT 42 AS answer")
result.show()

# Primeira query
r1 = duckdb.sql("SELECT 42 AS i")

# Segunda query referencia r1
duckdb.sql("SELECT i * 2 AS k FROM r1").show()

---

## 📂 Bloco 2: Leitura de Arquivos

DuckDB pode ler diversos formatos de arquivo diretamente.

In [ ]:
import duckdb

# CSV - Usando Relation
rel = duckdb.read_csv("example.csv")
print(rel)

# CSV - Query direta
result = duckdb.sql("SELECT * FROM 'example.csv'")
result.show()

# Parquet
rel = duckdb.read_parquet("example.parquet")
duckdb.sql("SELECT * FROM 'example.parquet'").show()

# JSON
rel = duckdb.read_json("example.json")
duckdb.sql("SELECT * FROM 'example.json'").show()

---

## 🐼 Bloco 3: Integração com DataFrames

DuckDB pode consultar DataFrames Pandas, Polars e Arrow Tables diretamente.

In [ ]:
import duckdb
import pandas as pd
import polars as pl
import pyarrow as pa

# Pandas DataFrame
pandas_df = pd.DataFrame({"a": [42]})
duckdb.sql("SELECT * FROM pandas_df").show()

# Polars DataFrame
polars_df = pl.DataFrame({"a": [42]})
duckdb.sql("SELECT * FROM polars_df").show()

# Arrow Table
arrow_table = pa.Table.from_pydict({"a": [42]})
duckdb.sql("SELECT * FROM arrow_table").show()

---

## 🔄 Bloco 4: Formatos de Saída

Convertendo resultados DuckDB para diferentes formatos Python.

In [ ]:
import duckdb

result = duckdb.sql("SELECT 42 AS num, 'hello' AS text")

# Lista de tuplas
rows = result.fetchall()
print("Tuplas:", rows)

# Pandas DataFrame
df = result.df()
print("\nPandas DataFrame:")
print(type(df))
print(df)

# Polars DataFrame  
result = duckdb.sql("SELECT 42 AS num, 'hello' AS text")
pl_df = result.pl()
print("\nPolars DataFrame:", type(pl_df))

# Arrow Table
result = duckdb.sql("SELECT 42 AS num, 'hello' AS text")
arrow = result.arrow()
print("Arrow Table:", type(arrow))

# NumPy
result = duckdb.sql("SELECT 42 AS num, 100 AS total")
numpy_result = result.fetchnumpy()
print("\nNumPy:", numpy_result)

---

## 💾 Bloco 5: Exportação de Dados

Salvando resultados em diferentes formatos de arquivo.

In [ ]:
import duckdb

# Parquet
duckdb.sql("SELECT 42 AS num").write_parquet("out.parquet")
# Ou usando COPY
duckdb.sql("COPY (SELECT 42) TO 'out.parquet'")

# CSV
duckdb.sql("SELECT 42 AS num").write_csv("out.csv")

# JSON
duckdb.sql("SELECT 42 AS num").write_json("out.json")

print("✅ Arquivos exportados com sucesso!")

---

## 🔌 Bloco 6: Gerenciamento de Conexões

Trabalhando com conexões persistentes e em memória.

In [ ]:
import duckdb

# Conexão em memória
con = duckdb.connect()
con.sql("SELECT 42").show()
con.close()

# Conexão persistente
con = duckdb.connect("file.db")
con.sql("CREATE TABLE IF NOT EXISTS test (i INTEGER)")
con.sql("INSERT INTO test VALUES (42)")
con.table("test").show()
con.close()

# Context manager (recomendado)
with duckdb.connect("file.db") as con:
    con.sql("SELECT * FROM test").show()
# Conexão fechada automaticamente

# Configurações customizadas
con = duckdb.connect(config={
    'threads': 4,
    'max_memory': '4GB',
    'default_order': 'DESC'
})
print("✅ Conexão configurada")

---

## 📊 Bloco 7: Caso de Uso - Análise de Vendas

Exemplo prático de análise de dados com Pandas + DuckDB.

In [ ]:
import duckdb
import pandas as pd

# Criar dados de exemplo
vendas = pd.DataFrame({
    'data': pd.date_range('2024-01-01', periods=100),
    'produto': ['A', 'B', 'C'] * 33 + ['A'],
    'quantidade': range(1, 101),
    'preco': [10.5, 20.0, 15.75] * 33 + [10.5]
})

# Análise usando DuckDB diretamente no DataFrame
resultado = duckdb.sql("""
    SELECT
        produto,
        COUNT(*) as total_vendas,
        SUM(quantidade) as total_quantidade,
        SUM(quantidade * preco) as receita_total,
        AVG(preco) as preco_medio
    FROM vendas
    GROUP BY produto
    ORDER BY receita_total DESC
""").df()

print(resultado)

---

## ⚡ Bloco 8: Boas Práticas e Performance

Dicas para otimizar o uso do DuckDB em Python.

In [ ]:
import duckdb
from concurrent.futures import ThreadPoolExecutor

# ✅ BOM: Reutilizar conexão
print("✅ Boa prática: Reutilizar conexão")
con = duckdb.connect()
for i in range(5):
    con.sql(f"SELECT {i}").fetchall()
con.close()

# ✅ BOM: Cada thread tem sua própria conexão
def boa_pratica_threads(thread_id):
    con = duckdb.connect('database.db')
    result = con.sql(f"SELECT {thread_id}").fetchall()
    con.close()
    return result

# ✅ BOM: Usar Parquet em vez de CSV
print("\n💡 Dica: Prefira Parquet para melhor performance")

# ✅ BOM: Prepared statements para queries repetidas
con = duckdb.connect()
stmt = con.prepare("SELECT $1 * $2")
result1 = stmt.execute([2, 3]).fetchall()
result2 = stmt.execute([5, 7]).fetchall()
print(f"Prepared statement: {result1}, {result2}")

print("\n✅ Boas práticas aplicadas!")

# Capitulo 06 Integracao Python

Notebook gerado automaticamente a partir do código fonte python.
